<a href="https://colab.research.google.com/github/parxec/Scraper-with-Selenium/blob/main/Scraping_Redcomputer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
#Installing the libraries!

!pip install BeautifulSoup4
!pip install pandas
from bs4 import BeautifulSoup
import requests
import pandas as pd

#Preparing list variables to store results
allproducts=[]
allprices=[]
allitems=[]
allsites=[]

#The following function will ask for the user to select one of 3 categories to extract.
# The return of this function is the category url section to be joined later to the base url.

def searchpage():
    
    try:
        import requests
        from bs4 import BeautifulSoup
        categories={'Electronics':1,'Fashion and Apparel':2,'Computers, Tablets and Accesories':3 }
        select= int(input(f'Select a number from the following categories: {categories}  '))

        if select == 1:
            category= f'electronics?cat=E10D24&r122_r1_r3_r1:page='
            print(f'Scraping Categorie:{list(categories.keys())[0]}')

        elif select == 2:
            category= f'fashion-%26-apparel?cat=F10D40&r122_r1_r3_r1:page='
            print(f'Scraping Categorie:{list(categories.keys())[1]}')    
        else:
            category= f'computers-tablets-%26-accessories?cat=C10D29&r122_r1_r3_r1:page='
            print(f'Scraping Categorie:{list(categories.keys())[2]}')

    except ValueError: 
        print('You entered a literal value, please, type a number instead, like 3')

    return category

    #In this part of the code we get the category url parse it to a variable soup.
#Also we obtain a calculation of pages in the category to extract all the data in each page.
#The pagination is particularly important because not all the categories have the same amount of pages.

category= searchpage()

url=f'https://www.pricesmart.com/site/do/en/category/{category}1&r122_r1_r3_r1:_sps=12'
print(url)
response= requests.get(url)
soup1= BeautifulSoup(response.content,'html.parser')
soup2= BeautifulSoup(soup1.prettify(), 'html.parser')


pagination= soup2.find_all('li')
for page in pagination:
    if '...' in page.text:
        pagination= int(page.text.replace('...','').strip())

#Here is the bulk of the work, where we first save the page as an html file.
#Then we extract the data from each tag and store it in the lists we saw at the begining.
#We could store all the webpages in a category localy by putting the code under the loop.
#However, saving all the pages would be a Waste of resources, unless ultimately required.

#With this piece of code we alert that this process can take some time.

pages=list(range(1,pagination+1))
print(f'Scraping {len(pages)} pages')
print('This process may take a few minutes, please be patient.')

with open(f'webpage.html','w',newline='',encoding='utf-8') as f:
    soup= BeautifulSoup(response.content,'html.parser')
    f.write(str(soup))
    f.close()
    
#This code prints a line to enhace the visual.
print()
print('________________________________________________________')


# For each page of a category we will extract all the specifed tags.
#Products, prices, availability, and description page url. 
for page in pages:
    
    url=f'https://www.pricesmart.com/site/do/en/category/{category}{page}&r122_r1_r3_r1:_sps=12'
    response= requests.get(url)
    soup= BeautifulSoup(response.content,'html.parser')

    #Extracts products titles.
    products= soup.find_all('div', class_='col-xs-12 col-sm-6 col-md-6 col-lg-3 px-3 px-sm-2 px-md-2 px-lg-2')
    for product in products:
        product= product.p.text.strip().replace('\n',',')
        allproducts.append(product)

    #Extracts prices.
    prices= soup.find_all(id='product-price', class_='currency')
    for price in prices:
        price= price.text.strip().replace('\n',',')
        allprices.append(float(price))

    #Extracts availability notice
    items_avail= soup.find_all('div', class_='search-product-box')
    for item in items_avail:
        item= item.i.text.strip().replace('\n',',')
        allitems.append(item)

    #Extracts product description page
    sites= soup.find_all('div', class_='col-xs-12 col-sm-6 col-md-6 col-lg-3 px-3 px-sm-2 px-md-2 px-lg-2')
    for site in sites:
        site= site.div.a['href'].strip()
        site2= 'https://www.pricesmart.com'+ site
        allsites.append(site2)
        

    #This line informs the amount of pages to scrape.
    print(f'Scraping page number:{page}')
    
    #we pass all the extacted data to a Pandas DataFrame and store it into a .csv file.
import pandas as pd



products_review = pd.DataFrame({'Products:':allproducts, 
                               'Price:':allprices, 
                               'Availability:':allitems,
                               'Website:':allsites})

products_review.to_csv(f'{category[:10]}.csv',index=False,encoding='utf-8',)

#Printing a line and an alert to inform the end of the process.
print('________________________________________________________')
print('The process has been completed!')

#Finally, we preview the results in the DataFrame.
products_review


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Select a number from the following categories: {'Electronics': 1, 'Fashion and Apparel': 2, 'Computers, Tablets and Accesories': 3}  1
Scraping Categorie:Electronics
https://www.pricesmart.com/site/do/en/category/electronics?cat=E10D24&r122_r1_r3_r1:page=1&r122_r1_r3_r1:_sps=12
Scraping 6 pages
This process may take a few minutes, please be patient.

________________________________________________________
Scraping page number:1
Scraping page number:2
Scraping page number:3
Scraping page number:4
Scraping page number:5
Scraping page number:6
________________________________________________________
The process has been completed!


,Products:,Price:,Availability:,Website:
0,Forza Voltage Regulator 600 W,1495.0,Available,https://www.pricesmart.com/site/do/en/pdp/360949
1,NexHT Wireless Security Camera 1080p Full HD,2695.0,Available,https://www.pricesmart.com/site/do/en/pdp/512090
2,Tzumi Aquaboost Boom Speaker with Bluetooth,2695.0,Available,https://www.pricesmart.com/site/do/en/pdp/423554
3,Hummer Bluetooth Speaker with LED Lights,3997.0,Available,https://www.pricesmart.com/site/do/en/pdp/420932
4,Tzumi Aura LED Universal Remote,775.0,Available,https://www.pricesmart.com/site/do/en/pdp/423471
...,...,...,...,...
57,"Hisense 55"" Smart Android 4K UHD TV 55A6G",39995.0,Out of stock in this club,https://www.pricesmart.com/site/do/en/pdp/413312
58,"Hisense 50"" Smart Android 4K UHD TV 50A6G",33990.0,Out of stock in this club,https://www.pricesmart.com/site/do/en/pdp/413295
59,Tylt XCELE 1 Plug-in Portable Battery,595.0,Out of stock in this club,https://www.pricesmart.com/site/do/en/pdp/445849
60,Swann Fourtify 4 Camera Perimeter Security System,10790.0,Out of stock in this club,https://www.pricesmart.com/site/do/en/pdp/443019


<Response [403]>
